# Code Explainer Assistant with LangGraph (Anthropic Version)

This notebook builds a chatbot that explains, debugs, and runs simple code snippets (Python-focused, with assembly notes). It uses:
- **State**: Tracks messages and code results.
- **Tools**: For code execution and basic syntax checks.
- **Router**: Decides if tool needed or direct explain.
- **Agent Loop**: ReAct-style for follow-ups (e.g., "fix this error").
- **Memory**: Persists across turns via checkpointer.

Test with queries like: "Explain this Python loop" or "Run: print(2+2)" or "Debug: for i in range(5): print(i/0)".


In [10]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Check if the key is loaded
print("Anthropic key loaded:", os.getenv("ANTHROPIC_API_KEY") is not None)


Anthropic key loaded: True


In [11]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_anthropic import ChatAnthropic  # Anthropic LLM
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.tools import tool
from langchain_experimental.utilities import PythonREPL  # For safe code execution
from operator import add
import operator

# Setup LLM with Anthropic (replace model if needed; requires ANTHROPIC_API_KEY env var)
llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0)
checkpointer = MemorySaver()  # Enables memory

# System prompt for code focus
system_prompt = SystemMessage(content="You are a code explainer. Explain clearly, suggest fixes for bugs, and use tools only when needed (e.g., for execution). Handle Python and basic assembly.")


## Step 1: Agent State

The state holds chat messages (appended via reducer) and a 'code_result' field to store outputs from tools, preventing overwrites.


In [12]:
class AgentState(TypedDict):
    messages: Annotated[list, add]  # Accumulates conversation
    code_result: Annotated[str, operator.add]  # Stores code outputs (string for simplicity)


## Step 2: Tools

- `run_python_code`: Safely executes Python snippets in a REPL (limited scope for safety).
- `explain_syntax`: A simple tool to break down code syntax (LLM-bound, but as a function for demo).

These are bound to the LLM so it decides when to call them.


In [13]:
# Tool 1: Execute Python code safely
python_repl = PythonREPL()

@tool
def run_python_code(code: str) -> str:
    """Run a Python code snippet and return the output or error."""
    try:
        result = python_repl.run(code)
        return f"Output: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Tool 2: Basic syntax explainer (custom function; LLM can call if needed)
@tool
def explain_syntax(code: str, language: str = "python") -> str:
    """Explain the syntax of a code snippet. Language: python or assembly."""
    if language == "assembly":
        return f"Assembly note: {code} - This is low-level; e.g., in MIPS, 'add $t0, $t1, $t2' adds registers."
    return f"Python syntax for '{code}': Breaks down structure, variables, and flow."

tools = [run_python_code, explain_syntax]
llm_with_tools = llm.bind_tools(tools)


## Step 3: Nodes

- `agent`: LLM processes input, decides on explanation or tool call.
- `tools`: Executes bound tools and updates state with results.


In [14]:
def agent(state: AgentState):
    # Prepend system prompt if first message
    if not state["messages"]:
        state["messages"].insert(0, system_prompt)

    # LLM invocation with tools (Anthropic handles tool calls)
    msg = llm_with_tools.invoke(state["messages"])
    return {
        "messages": [msg],
        "code_result": state.get("code_result", "")
    }

from langgraph.prebuilt import ToolNode
tool_node = ToolNode(tools)  # Auto-executes tool calls


## Step 4: Router (Conditional Edge)

Simple decision: If LLM output has tool calls, route to `tools`; else, end with explanation.


In [15]:
def should_continue(state: AgentState):
    last_msg = state["messages"][-1]
    if last_msg.tool_calls:
        return "tools"  # Continue to tools
    return END  # Final response (explanation)


## Step 5: Build the Graph

Chain: Entry to agent.
Router: Conditional from agent.
Loop: Tools back to agent (ReAct for iterations, e.g., debug loop).
Memory: Compile with checkpointer.


In [16]:
graph = StateGraph(state_schema=AgentState)

# Add nodes
graph.add_node("agent", agent)
graph.add_node("tools", tool_node)

# Edges
graph.set_entry_point("agent")
graph.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
graph.add_edge("tools", "agent")  # Loop for observation/refinement

# Compile with memory
app = graph.compile(checkpointer=checkpointer)


## Step 6: Usage

Invoke with a config (thread_id for memory). Test single-turn and multi-turn.

- Single: Explain a loop.
- Multi: Run code, then "What's wrong?" (remembers output).


In [18]:
# Config for session (use same thread_id for memory)
config = {"configurable": {"thread_id": "code_explain_session_1"}}

# Example 1: Simple explanation (no tool)
input_explain = {
    "messages": [HumanMessage(content="Explain: for i in range(3): print(i)")]
}
output_explain = app.invoke(input_explain, config)
print("Explanation:", output_explain["messages"][-1].content)

# Example 2: Run code (triggers tool)
input_run = {
    "messages": [HumanMessage(content="Run this Python: x = 5; y = x * 2; print(y)")]
}
output_run = app.invoke(input_run, config)
print("Run Result:", output_run["messages"][-1].content)
print("Stored Result:", output_run["code_result"])

# Example 3: Multi-turn debug (memory loads prior)
input_debug = {
    "messages": [HumanMessage(content="Debug why it errored and fix it.")]  # Refers to previous if any
}
output_debug = app.invoke(input_debug, config)
print("Debug:", output_debug["messages"][-1].content)

# Assembly example
input_assembly = {
    "messages": [HumanMessage(content="Explain MIPS assembly: add $t0, $t1, $t2")]
}
output_assembly = app.invoke(input_assembly, config)
print("Assembly Explain:", output_assembly["messages"][-1].content)


Explanation: Let me provide a detailed explanation of this Python code:

**`for i in range(3): print(i)`**

This is a **for loop** that consists of several parts:

1. **`for`** - The keyword that starts a for loop
2. **`i`** - The loop variable (iterator) that takes on different values in each iteration
3. **`in`** - Keyword indicating membership/iteration
4. **`range(3)`** - A function that generates a sequence of numbers from 0 to 2 (3 is exclusive)
5. **`:`** - Colon that marks the end of the for loop header
6. **`print(i)`** - The body of the loop that executes for each iteration

**How it works:**
- `range(3)` creates a sequence: [0, 1, 2]
- In the first iteration, `i = 0`, so `print(i)` outputs `0`
- In the second iteration, `i = 1`, so `print(i)` outputs `1`
- In the third iteration, `i = 2`, so `print(i)` outputs `2`
- After 3 iterations, the loop ends

**Output:**
```
0
1
2
```

This is a common pattern for repeating an action a specific number of times in Python.
Run Result: 